In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1003"

In [2]:
# 7 layers CNN pooling_stride = [2, 2, 2, 2, 2, 1, 1]

## Importing Libraries

In [3]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [4]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [5]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [6]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [7]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 7    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2, 1, 1]
weights_shape = [[5, n_variables, 32],
                 [5, 32, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64],
                 [5, 64, 64]]
bias_shape = [32, 64, 64, 64, 64, 64, 64]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [8]:
print pos_weight

3.18601822124


In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [11]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [12]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [13]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [14]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.316583, fmeasure_max= 0.434815, precision= 0.287182, recall= 0.894817
Train Random guess:  , auc= 0.259573fmeasure_max= 0.408214, precision= 0.256450, recall= 1.000000
Validation Iter 0, auc= 0.247542, fmeasure_max= 0.383872, precision= 0.238213, recall= 0.987993
Validation Random guess:  auc= 0.233120, fmeasure_max= 0.372642, precision= 0.231866, recall= 0.948542
Minibatch Loss= 1.092687
Tarin Iter 100, auc= 0.597315, fmeasure_max= 0.539841, precision= 0.631702, recall= 0.471304
Train Random guess:  , auc= 0.236898fmeasure_max= 0.368185, precision= 0.225718, recall= 0.998261
Validation Iter 100, auc= 0.695734, fmeasure_max= 0.635063, precision= 0.663551, recall= 0.608919
Validation Random guess:  auc= 0.249762, fmeasure_max= 0.381944, precision= 0.239443, recall= 0.943396
Minibatch Loss= 0.923534
Tarin Iter 200, auc= 0.800689, fmeasure_max= 0.740465, precision= 0.750943, recall= 0.730275
Train Random guess:  , auc= 0.223538fmeasure_max= 0.351160, precision= 0.2129

In [15]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

ValueError: Parent directory of ../model/CNN1003/CNN1003 doesn't exist, can't save.

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [16]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.845090 F1_max = 0.788869 prec = 0.794889 rec = 0.782939
Test Random guess: auc= 0.231657 fmeasure_max= 0.375694 , precision= 0.231295 , recall= 1.000000


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})